In [1]:
from gensim.models import KeyedVectors

In [2]:
file_name = "jawiki.all_vectors.200d-002.txt"
model = KeyedVectors.load_word2vec_format(file_name)

In [1]:
import pickle

In [4]:
with open('word2vec_model.pickle', mode='wb') as f:
    pickle.dump(model, f)

In [2]:
with open('word2vec_model.pickle', mode='rb') as f:
    test_pickle_model = pickle.load(f)

In [22]:
import numpy as np
import mojimoji
import MeCab as mc

m = mc.Tagger("-Ochasen")

In [16]:
def mecab_tokenizer(text: str):
    """
    テキストを分かち書きするメソッド
    :param text: 分割したいテキスト
    :return: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

In [17]:
def vectorize_word(text: str):
    """
    元word_analysis()
    ベクトル化を行うメソッド
    :param text: ベクトル化するテキスト
    :return: ベクトル化したテキスト(array型)
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """
    cos類似度を計算
    :param v1: word2vecによりベクトル化したテキスト1(ndarray型)
    :param v2: word2vecによりベクトル化したテキスト2(ndarray型)
    :return: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [18]:
def clean_text(text: str):
    """
    テキストの正規化
    :param text: 正規化するテキスト
    :return: 正規化後のテキスト
    """
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)  # 半角文字を全角文字に統一(数字, 英語以外)
    text = mojimoji.zen_to_han(text, kana=False)  # 全角文字を半角文字に統一(かな以外)
    text = text.lower()  # 小文字に統一
    return text

In [19]:
test_word = '縣体協の學 童相撲人會'

In [25]:
def vectorize_word_pickle(text: str):
    """
    元word_analysis()
    ベクトル化を行うメソッド
    :param text: ベクトル化するテキスト
    :return: ベクトル化したテキスト(array型)
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(test_pickle_model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """
    cos類似度を計算
    :param v1: word2vecによりベクトル化したテキスト1(ndarray型)
    :param v2: word2vecによりベクトル化したテキスト2(ndarray型)
    :return: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [23]:
origin_vector = vectorize_word(test_word)

In [27]:
origin_vector

array([ 0.05823259, -0.16913912, -0.05173661, -0.17274819, -0.0403929 ,
       -0.03585067,  0.19111562,  0.13368553, -0.25446417,  0.08546442,
       -0.09687355, -0.33496718, -0.11772048,  0.02910109,  0.14799757,
       -0.20030577,  0.13155583, -0.11711848, -0.07376874, -0.10243418,
        0.11118599,  0.01274949, -0.59391406,  0.02399342,  0.09026742,
       -0.03922956, -0.11410013,  0.14471516, -0.10006444, -0.11508192,
        0.11593011, -0.13577288,  0.14044928, -0.08384098,  0.08344796,
       -0.04024466, -0.25533674,  0.29996674, -0.37464159, -0.15103976,
       -0.10001664,  0.32393306, -0.00280211,  0.05887438, -0.07633487,
       -0.01925723,  0.43012448, -0.37823561, -0.08142076,  0.08572617,
       -0.23463002, -0.01279685,  0.07153545,  0.04136499, -0.11128016,
       -0.19464023,  0.06922324, -0.04747667, -0.04059302, -0.1252287 ,
        0.12948431, -0.30929148,  0.0494603 , -0.11413988,  0.22758553,
       -0.16626524, -0.01561721, -0.26964435,  0.28762528,  0.21

In [26]:
pickle_vector = vectorize_word_pickle(test_word)

In [28]:
pickle_vector

array([ 0.05823259, -0.16913912, -0.05173661, -0.17274819, -0.0403929 ,
       -0.03585067,  0.19111562,  0.13368553, -0.25446417,  0.08546442,
       -0.09687355, -0.33496718, -0.11772048,  0.02910109,  0.14799757,
       -0.20030577,  0.13155583, -0.11711848, -0.07376874, -0.10243418,
        0.11118599,  0.01274949, -0.59391406,  0.02399342,  0.09026742,
       -0.03922956, -0.11410013,  0.14471516, -0.10006444, -0.11508192,
        0.11593011, -0.13577288,  0.14044928, -0.08384098,  0.08344796,
       -0.04024466, -0.25533674,  0.29996674, -0.37464159, -0.15103976,
       -0.10001664,  0.32393306, -0.00280211,  0.05887438, -0.07633487,
       -0.01925723,  0.43012448, -0.37823561, -0.08142076,  0.08572617,
       -0.23463002, -0.01279685,  0.07153545,  0.04136499, -0.11128016,
       -0.19464023,  0.06922324, -0.04747667, -0.04059302, -0.1252287 ,
        0.12948431, -0.30929148,  0.0494603 , -0.11413988,  0.22758553,
       -0.16626524, -0.01561721, -0.26964435,  0.28762528,  0.21

In [29]:
np.array_equal(origin_vector, pickle_vector)

True

### 読み込み解決したわ…